In [3]:
import pandas as pd
from selenium import webdriver
import time
import os
import re
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from sqlalchemy import true
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import json

scopes = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name("/Users/ryoaraki/Downloads/avid-pager-420607-47f0563c7b60.json", scopes) #access the json key you downloaded earlier 
file = gspread.authorize(credentials) # authenticate the JSON key with gspread
sheet = file.open("resell list") #open sheet
sheetL = sheet.sheet1

skus = sheetL.col_values(7)[1:] 
SKUs = [str(x) for x in skus if x != '']  # Use list comprehension to filter out empty values

data=[]

for sku in SKUs:
    url = "https://www.carousell.com.hk/search/" + sku
    browser = webdriver.Chrome()
    time.sleep(3)

    # ブラウザで検索
    browser.get(url)
    time.sleep(3)
    def scroll_to_bottom():
        actions = ActionChains(browser)
        actions.send_keys(Keys.END).perform()
    for _ in range(5):  # Adjust the number of scrolls as needed
        scroll_to_bottom()
        time.sleep(5) 

    # 商品ごとのHTMLを全取得
    soup = bs(browser.page_source, "html.parser")
    posts = soup.find_all("div",{"class":"D_pm M_kS"})

    # 商品ごとに名前と値段、購入済みかどうか、URLを取得
    for post in posts:    
        # Assuming 'element' is the WebElement representing the div
        p_tag = post.find("p", class_="D_mA M_hE D_mB M_hF D_mG M_hJ D_mI M_hL D_mM M_hP D_mP M_hS D_mZ")
        if p_tag:
            price = float(p_tag.get_text().replace("HK$", '').replace(",", '').strip())
            print(price)
        else: continue
        
        data.append({"SKU": sku, "Price": price, "Count": 1})

    browser.close()

df = pd.DataFrame(data)
df = df.groupby('SKU').agg({'Price':['min','mean','max'],'Count':'sum'}).reset_index()

print(df)
        
# Export to Gsheet
sheetL.update('I2', df.values.tolist(), value_input_option='RAW')

70.0
265.0
9800.0
199.0
80.0
100.0
1.0
2190.0
2249.0
532.0
578.0


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=124.0.6367.60)
Stacktrace:
0   chromedriver                        0x00000001052f6934 chromedriver + 4368692
1   chromedriver                        0x00000001052eedc8 chromedriver + 4337096
2   chromedriver                        0x0000000104f12c04 chromedriver + 289796
3   chromedriver                        0x0000000104efbd00 chromedriver + 195840
4   chromedriver                        0x0000000104efbc3c chromedriver + 195644
5   chromedriver                        0x0000000104f8cd24 chromedriver + 789796
6   chromedriver                        0x0000000104f49ab4 chromedriver + 514740
7   chromedriver                        0x0000000104f4a50c chromedriver + 517388
8   chromedriver                        0x00000001052bae50 chromedriver + 4124240
9   chromedriver                        0x00000001052bfc40 chromed